<h1 align=center><font size = 5>Finding the right spot for a burger bar in Budapest</font></h1>

<h5 align=center><font size = 5>Import necessary libs</font></h5>

In [2]:
#!conda install -c conda-forge beautifulsoup4 --yes
#!conda install -c conda-forge geocoder --yes
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0 --yes

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

import requests
import urllib
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from pandas.io.json import json_normalize

print('Libraries imported.')

Libraries imported.


<h5 align=center><font size = 5>Load neighborhood infos from Wikipedia</font></h5>

In [3]:
URL = "https://hu.wikipedia.org/wiki/Budapest_v%C3%A1rosr%C3%A9szeinek_list%C3%A1ja"
html_doc = requests.get(URL).text

#Load HTML
soup = BeautifulSoup(html_doc, 'html.parser')

#Extract the table object
table = soup.table

#Extract the table header
header = [e.text.replace('\n','') for e in table.find_all('th')]

#Extract texts from table rows
rows = []
table_rows = table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.replace('\n','') for tr in td]
    if len(row) > 0:
        row[3] = row[3].replace(u'\xa0','')
        #exclude neighborhoods with small or unknown populations
        if row[3] != '—' and row[3] != '' and int(row[3]) >= 1000:
            #data cleansing
            row[2] =  row[2][9:-1]
            if row[2] == 'I. és XII. kerület':
                row[2] = 'I. kerület'
            if row[2] == 'II. és XII. kerület':
                row[2] = 'XII. kerület'
            if row[2] == 'I. és XI. kerület':
                row[2] = 'XI. kerület'
            if row[2] == 'IV. és XIII. kerület':
                row[2] = 'XIII. kerület'
            if row[2] == 'XI. és XII. kerület':
                row[2] = 'XI. kerület'
            if row[2] == 'I. és II. kerület':
                row[2] = 'I. kerület'
            if row[2] == 'II. és III. kerület':
                row[2] = 'III. kerület'
            row[3] = int(row[3])
            rows.append(row)

#Convert list into dataframe
rows_df = pd.DataFrame(rows, columns=header)[["Név","Kerület","Lakosság (2001)"]]
rows_df.columns = ["Neighborhood","District","Population"]
rows_df.head()

,Neighborhood,District,Population
0,Akadémiaújtelep,XVII. kerület,2895
1,Albertfalva,XI. kerület,11845
2,Alsórákos,XIV. kerület,29023
3,Angyalföld,XIII. kerület,62006
4,Árpádföld,XVI. kerület,6186


<h5 align=center><font size = 5>Append geolocation informations to the dataframe</font></h5>

In [4]:
#Query location infos from openstreetmap API
def get_coords_local(nbh, distr, output_as='center'):
    """
    Parameters
    ----------
    nbh : str
        name of the neighborhood in english and lowercase
    distr: str
        name of the distrinct in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             nbh+', '+distr+', Budapest, Közép-Magyarország, Magyarország',
                             '&format=json&polygon=0')
    #print(url)
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [5]:
nbh_df = rows_df.copy()

ltd = []
lng = []
for index, row in nbh_df.iterrows():
    print(row[0])
    lat, long = get_coords_local(nbh=row[0], distr=row[1], output_as='center')
    ltd.append(lat)
    lng.append(long)

nbh_df['Latitude'] = ltd
nbh_df['Longitude'] = lng

print(nbh_df.shape)
nbh_df.head()

Akadémiaújtelep
Albertfalva
Alsórákos
Angyalföld
Árpádföld
Baross Gábor-telep
Békásmegyer
Bélatelep
Belsőmajor
Belváros
Bókaytelep
Budafok
Budaliget
Budatétény
Cinkota
Csatárka
Csepel-Belváros
Csepel-Csillagtelep
Csepel-Erdőalja
Csepel-Erdősor
Csepel-Gyártelep
Csepel-Háros
Csepel-Kertváros
Csepel-Királyerdő
Csepel-Királymajor
Csepel-Szabótelep
Csillaghegy
Erdőskert
Erzsébetfalva
Erzsébettelep
Erzsébetváros
Farkasvölgy
Felhévíz
Felsőrákos
Ganzkertváros
Ganztelep
Gazdagrét
Gellérthegy
Gloriett-telep
Gubacs
Gubacsipuszta
Gyárdűlő
Hársakalja
Havanna-lakótelep
Herminamező
Hűvösvölgy
Istenhegy
Istvánmező
Istvántelek
József Attila-lakótelep
Káposztásmegyer
Kaszásdűlő
Kelenföld
Kelenvölgy
Kerepesdűlő
Keresztúridűlő
Kispest
Kissvábhegy
Kiszugló
Kossuth Ferenc-telep
Kossuthfalva
Kőbánya-Kertváros
Krepuska Géza-telep
Krisztinaváros
Kurucles
Kútvölgy
Lágymányos
Lakatos-lakótelep
Laposdűlő
Ligettelek
Lipótváros
Liptáktelep
Lónyaytelep
Madárdomb
Máriaremete
Mártonhegy
Mátyásföld
Megyer
Miklóstelep
M

,Neighborhood,District,Population,Latitude,Longitude
0,Akadémiaújtelep,XVII. kerület,2895,47.481613,19.219526
1,Albertfalva,XI. kerület,11845,47.447294,19.034295
2,Alsórákos,XIV. kerület,29023,47.525178,19.128280
3,Angyalföld,XIII. kerület,62006,47.538369,19.082082
4,Árpádföld,XVI. kerület,6186,47.533133,19.198560


<h5 align=center><font size = 5>Show map of Budapest with its neighborhoods</font></h5>

In [6]:
address = 'Budapest, Hungary'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
BP_ltd = location.latitude
BP_lng = location.longitude
print('The coordinates of Budapest are {}, {}.'.format(BP_ltd, BP_lng))

The coordinates of Budapest are 47.4983815, 19.0404707.


In [145]:
# create map of Budapest using latitude and longitude values
map_BP = folium.Map(location=[BP_ltd, BP_lng], zoom_start=12)

# add markers to map
for lat, lng, local in zip(nbh_df['Latitude'], nbh_df['Longitude'], nbh_df['Neighborhood']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_BP)  

map_BP_sample = map_BP    

map_BP

<h5 align=center><font size = 5>Query venue infos from Forsquare</font></h5>

In [8]:
#function to get all venues for a neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500, categoryIds='', query='', type='search'):
    try:
        venues_list=[]
        col_names = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','name','lat','lng','categories']
        venues_df = pd.DataFrame(columns=col_names)
        
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/**--type_substring--**?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            if (query != ''):
                url = url + '&q={}'
                url = url.format(query)
            
            if type == 'explore':
                url = url + '&section={}'
                url = url.format('sights')
            
            url = url.replace('**--type_substring--**',type)
            
            # make the GET request
            response = requests.get(url).json()
            
            if type == 'search':
                results = response["response"]['venues']
            else:
                results = response['response']['groups'][0]['items']

            # return only relevant information for each nearby venue
            
            if type == 'explore':
                dataframe = json_normalize(results) # flatten JSON
                
                #check if the query returns any venue
                if len(dataframe.index) > 0:
                    # filter columns
                    filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
                    #print(filtered_columns)
                    dataframe_filtered = dataframe.loc[:, filtered_columns]

                    # filter the category for each row
                    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

                    # clean columns
                    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

                    dataframe_filtered = dataframe_filtered[['name','lat','lng','categories']]
                    dataframe_filtered['Neighborhood'] = name
                    dataframe_filtered['Neighborhood Latitude'] = lat
                    dataframe_filtered['Neighborhood Longitude'] = lng

                    #reorder columns
                    dataframe_filtered = dataframe_filtered[col_names]

                    venues_df = pd.concat([venues_df, dataframe_filtered])
                #else:
                #    nearby_venues = pd.DataFrame(columns=col_names)
            else:
                for v in results:
                    success = False
                    try:
                        category = v['categories'][0]['name']
                        success = True
                    except:
                        pass

                    if success:
                        venues_list.append([(
                            name, 
                            lat, 
                            lng, 
                            v['name'], 
                            v['location']['lat'], 
                            v['location']['lng'],
                            v['categories'][0]['name']
                        )])

        if type == 'explore':
            nearby_venues = venues_df
        else:
            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [71]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = 'YXUBI3K2DD1KAV1MQOKMKL30R3K20YXGKW2R2EJY12JTJT23'
CLIENT_SECRET = 'XVBSAKPRRKEIQAW2QLJC0GM2GH21KA02KBYZV5JDGIQMEK0J'
VERSION = '20181020'

office_id = '4bf58dd8d48988d124941735' # Office
uni_id = '4bf58dd8d48988d198941735,4bf58dd8d48988d1ae941735' # College Academic Building, University
ffr_id = '4bf58dd8d48988d16e941735' # Fast Food Restaurant
burg_id = '4bf58dd8d48988d16c941735' # Burger Joint
# tourist attractions can we query with the following search text: tourist

In [10]:
#fast food restaurant
BP_ffr = getNearbyVenues(names=nbh_df['Neighborhood'], latitudes=nbh_df['Latitude'], longitudes=nbh_df['Longitude'], radius=radius, categoryIds=ffr_id+','+burg_id)
print(BP_ffr.shape)
BP_ffr.head()

(307, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akadémiaújtelep,47.481613,19.219526,Heppi Hurka falatozó,47.485454,19.217227,Fast Food Restaurant
1,Albertfalva,47.447294,19.034295,Pavilon Bufe,47.444100,19.037325,Burger Joint
2,Albertfalva,47.447294,19.034295,Messze van de van kaja,47.450206,19.030176,Burger Joint
3,Angyalföld,47.538369,19.082082,Garage Food&Grill,47.539428,19.076721,Burger Joint
4,Angyalföld,47.538369,19.082082,Mecca Fit Food,47.534478,19.082696,Fast Food Restaurant


In [11]:
#offices
BP_office = getNearbyVenues(names=nbh_df['Neighborhood'], latitudes=nbh_df['Latitude'], longitudes=nbh_df['Longitude'], radius=radius, categoryIds=office_id)
print(BP_office.shape)
BP_office.head()

(2128, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akadémiaújtelep,47.481613,19.219526,Konkam Studio,47.482298,19.220110,Coworking Space
1,Akadémiaújtelep,47.481613,19.219526,Könyvelő,47.485447,19.219976,Office
2,Akadémiaújtelep,47.481613,19.219526,Könyvelő,47.485278,19.220253,Office
3,Albertfalva,47.447294,19.034295,Mixpakk Kft.,47.443897,19.036158,Office
4,Albertfalva,47.447294,19.034295,BFS consulting,47.450162,19.031822,Coworking Space


In [72]:
#universities & colleges
BP_uni = getNearbyVenues(names=nbh_df['Neighborhood'], latitudes=nbh_df['Latitude'], longitudes=nbh_df['Longitude'], radius=radius, categoryIds=uni_id)
print(BP_uni.shape)
BP_uni.head()

(395, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alsórákos,47.525178,19.128280,METU Egressy Campus,47.522093,19.129243,College Academic Building
1,Alsórákos,47.525178,19.128280,Budapesti Innovatív Középiskola,47.521923,19.129431,General College & University
2,Angyalföld,47.538369,19.082082,Kodolányi János Egyetem,47.534205,19.080793,University
3,Angyalföld,47.538369,19.082082,Kodolányi János Főiskola,47.534083,19.081262,University
4,Belváros,47.491918,19.054694,Sapientia Szerzetesi Hittudományi Főiskola,47.492779,19.052928,University


In [79]:
#explore tourist attractions
BP_tour_attr = getNearbyVenues(names=nbh_df['Neighborhood'], latitudes=nbh_df['Latitude'], longitudes=nbh_df['Longitude'], radius=radius, query='tourist', type='explore')
print(BP_tour_attr.shape)
BP_tour_attr.head()

(1869, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akadémiaújtelep,47.481613,19.219526,525 Tér,47.480901,19.219802,Park
1,Akadémiaújtelep,47.481613,19.219526,Akadémia Patika,47.480657,19.222662,Pharmacy
2,Akadémiaújtelep,47.481613,19.219526,Pékség,47.485254,19.216986,Bakery
3,Akadémiaújtelep,47.481613,19.219526,Heppi Hurka falatozó,47.485454,19.217227,Fast Food Restaurant
0,Albertfalva,47.447294,19.034295,VakVarjú Vendéglő és Sörkert,47.449676,19.036447,Hungarian Restaurant


In [92]:
#clean tourist venues
#BP_tour_attr.head(100)
#exclude the following categories
cat_list = ['Pharmacy','Bakery','Fast Food Restaurant','Hungarian Restaurant','Pub','Ice Cream Shop','Playground',
           'Grocery Store','Laundromat','Food Court','Lottery Retailer','Dessert Shop','College Cafeteria','Chinese Restaurant',
           'Spa','Electronics Store','Food & Drink Shop','Music Store','Supermarket','Restaurant','Cosmetics Shop','Hobby Shop',
           'Bar','Paper / Office Supplies Store','Brewery','Cocktail Bar','Park','Theater','Hotel','Café','Burger Joint',
           'Coffee Shop','Boutique','Asian Restaurant','Wine Bar','Thai Restaurant','Vegetarian / Vegan Restaurant',
           'Gift Shop','Indian Restaurant','Tea Room','Steakhouse','Gluten-free Restaurant','Liquor Store','Greek Restaurant',
           'French Restaurant','Bistro','Candy Store','Sporting Goods Shop','Flower Shop','Eastern European Restaurant',
           'Bubble Tea Shop','Italian Restaurant','Falafel Restaurant','Pizza Place','Mediterranean Restaurant',
           'Salad Place','Pet Café','Sushi Restaurant','Beer Bar','Bookstore','Shoe Store','Breakfast Spot',
           'Food Truck','Smoke Shop','Optical Shop','Middle Eastern Restaurant','Beer Garden','Pet Store','Food',
           'Farmers Market','American Restaurant','Bowling Alley','Diner','Print Shop','BBQ Joint',
           'Lounge','Turkish Restaurant', 'Sports Bar', 'Paintball Field', 'Squash Court', 'Photography Studio',
           'Harbor / Marina', 'Motorcycle Shop', 'Vietnamese Restaurant', 'Automotive Shop', 'Deli / Bodega',
            'Furniture / Home Store', 'Sandwich Place', 'Dive Bar', 'Record Shop', 'Gourmet Shop', 'Tapas Restaurant', 
            'Indoor Play Area', 'Dog Run', 'Fruit & Vegetable Store', 'Mountain', 'Tennis Court', 'Fishing Spot', 
            'Yoga Studio', 'Pastry Shop', 'Buffet', 'Department Store', 'Shopping Mall', 'Massage Studio', 'Wine Shop',
            'Neighborhood', 'Auto Dealership', 'Butcher', 'Nightclub', 'Pool Hall', 'Snack Place', 'Cafeteria', 
            'Clothing Store', 'Auto Workshop', 'Doner Restaurant', 'College Rec Center', 'Frozen Yogurt Shop', 
            'Toy / Game Store', 'Perfume Shop', 'Creperie', 'Korean Restaurant', 'Kids Store', 'Burrito Place', 
            'Office', "Women's Store", 'Chocolate Shop', 'Modern European Restaurant', 'Cupcake Shop',
            'Mexican Restaurant', 'Gastropub', 'College Soccer Field', 'Donut Shop', 'Salon / Barbershop',
            'Convenience Store', 'College Track', 'Fried Chicken Joint', 'Health & Beauty Service', 'Noodle House', 
            'Russian Restaurant', 'Japanese Restaurant', 'Bagel Shop', 'General Entertainment', 'Hardware Store', 
            'Tattoo Parlor', 'Gaming Cafe', 'Taco Place', 'Soup Place', 'Gym Pool', 'Medical Center',
            'Whisky Bar', 'Baseball Field', 'Field', 'German Restaurant', 'Fish Market', 'Miscellaneous Shop', 
            'Climbing Gym', 'Shop & Service', 'Swiss Restaurant', 'Thrift / Vintage Store', 'Design Studio',
            'Fish & Chips Shop', 'Road', 'Health Food Store', 'Video Game Store', 'Cheese Shop', 'Dance Studio',
            'Other Repair Shop', 'Martial Arts Dojo', 'Dairy Store', 'Basketball Court', 'Flea Market', 'Garden Center',
            'Video Store', 'Skate Park', 'Stationery Store', 'Nursery School', 'Jewelry Store', 'Pie Shop', 
            'Arts & Crafts Store', 'Student Center', 'Shoe Repair', 'Outdoor Supply Store', 'Caucasian Restaurant', 
            'Tanning Salon', 'Beer Store', 'Dim Sum Restaurant', 'Arcade', 'Seafood Restaurant', 'Hookah Bar', 'Hotel Bar']

BP_tour_attr_is = ~BP_tour_attr['Venue Category'].isin(cat_list)
BP_tour_attr_filt = BP_tour_attr[BP_tour_attr_is]
print(BP_tour_attr[BP_tour_attr_is].shape)
BP_tour_attr_filt.head()

(290, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,Albertfalva,47.447294,19.034295,Nyéki Imre Uszoda,47.451057,19.036428,Pool
1,Alsórákos,47.525178,19.128280,Babos Siiskola,47.522512,19.128897,Ski Area
2,Alsórákos,47.525178,19.128280,Rákosmezei Tér,47.524486,19.132315,Plaza
1,Angyalföld,47.538369,19.082082,Oxygen Wellness,47.535743,19.080129,Gym / Fitness Center
2,Angyalföld,47.538369,19.082082,Vasas pálya,47.537688,19.081192,Soccer Field


In [94]:
# function to add markers for given type of venue to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=1,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [146]:
#show samples of every type of venues
addToMap(BP_ffr.sample(n=100), 'red', map_BP_sample) #add fast food restaurants
addToMap(BP_office.sample(n=100), 'yellow', map_BP_sample) #add offices
addToMap(BP_uni.sample(n=100), 'green', map_BP_sample) #add schools
addToMap(BP_tour_attr_filt.sample(n=100), 'purple', map_BP_sample) #add tourist places

map_BP_sample

<h5 align=center><font size = 5>Count venues for all neighborhoods</font></h5>

In [141]:
BP_ffr_cnt = BP_ffr[['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']].groupby(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']).size().reset_index(name='FFR counts')
BP_office_cnt = BP_office[['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']].groupby(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']).size().reset_index(name='Office counts')
BP_uni_cnt = BP_uni[['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']].groupby(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']).size().reset_index(name='Uni counts')
BP_tour_attr_cnt = BP_tour_attr_filt[['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']].groupby(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']).size().reset_index(name='Tourist spot counts')

#join aggregated data
BP_neighborhoods = nbh_df
BP_neighborhoods = BP_neighborhoods.join(BP_ffr_cnt[['Neighborhood','FFR counts']].set_index('Neighborhood'), on='Neighborhood')
BP_neighborhoods = BP_neighborhoods.join(BP_office_cnt[['Neighborhood','Office counts']].set_index('Neighborhood'), on='Neighborhood')
BP_neighborhoods = BP_neighborhoods.join(BP_uni_cnt[['Neighborhood','Uni counts']].set_index('Neighborhood'), on='Neighborhood')
BP_neighborhoods = BP_neighborhoods.join(BP_tour_attr_cnt[['Neighborhood','Tourist spot counts']].set_index('Neighborhood'), on='Neighborhood')
BP_neighborhoods = BP_neighborhoods.fillna(0)

<h5 align=center><font size = 5>Set the weights for all type of venues and calculate the fitness for all neighborhoods</font></h5>

In [143]:
# weight to calculate the population equivalent of the venues
weight_tourist = 5000
weight_uni = 2500
weight_offices = 500

#calculate fitness
BP_neighborhoods['Fitness'] = (BP_neighborhoods['Population'] + BP_neighborhoods['Office counts']*weight_offices + BP_neighborhoods['Uni counts']*weight_uni + BP_neighborhoods['Tourist spot counts']*weight_tourist) / (BP_neighborhoods['FFR counts'] + 1)
#sort
BP_neighborhoods = BP_neighborhoods.sort_values(by=['Fitness'], ascending = False)

BP_neighborhoods.head()

,Neighborhood,District,Population,Latitude,Longitude,FFR counts,Office counts,Uni counts,Tourist spot counts,Fitness
52,Kelenföld,XI. kerület,53332,47.465171,19.041108,1.0,50.0,3.0,6.0,57916.0
117,Szemlőhegy,II. kerület,2753,47.524329,19.028179,0.0,45.0,1.0,6.0,57753.0
126,Törökőr,XIV. kerület,15170,47.507438,19.111503,0.0,43.0,2.0,3.0,56670.0
56,Kispest,XIX. kerület,52349,47.448441,19.146518,0.0,7.0,0.0,0.0,55849.0
44,Herminamező,XIV. kerület,12264,47.518684,19.099723,0.0,44.0,2.0,3.0,54264.0


<h5 align=center><font size = 5>Show the best neighborhood Kelenföld with all its venues</font></h5>

In [144]:
# create map of Kelenföld using latitude and longitude values
kelen_ltd = float(BP_neighborhoods.head(1)['Latitude'])
kelen_lng = float(BP_neighborhoods.head(1)['Longitude'])

map_kelen = folium.Map(location=[kelen_ltd, kelen_lng], zoom_start=14)

# add markers to neighborhood
for lat, lng, local in zip(BP_neighborhoods.head(1)['Latitude'], BP_neighborhoods.head(1)['Longitude'], BP_neighborhoods.head(1)['Neighborhood']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kelen)  

#add venues
addToMap(BP_ffr[BP_ffr['Neighborhood'] == 'Kelenföld'], 'red', map_kelen) #add fast food restaurants
addToMap(BP_office[BP_office['Neighborhood'] == 'Kelenföld'], 'yellow', map_kelen) #add offices
addToMap(BP_uni[BP_uni['Neighborhood'] == 'Kelenföld'], 'green', map_kelen) #add schools
addToMap(BP_tour_attr_filt[BP_tour_attr_filt['Neighborhood'] == 'Kelenföld'], 'purple', map_kelen) #add tourist places
    
map_kelen